In [1]:
import pyspark
from pyspark.sql import SparkSession
import findspark
findspark.init()
findspark.find()
spark=SparkSession.builder.appName("jupyter").getOrCreate()



[['2023-01-01', '1:05:00', '', '', '', '', '', 'U1'],
 ['2023-01-01', '1:25:00', '', '', '', '', '', 'U1'],
 ['2023-01-02', '2:05:00', '', '', '', '', '', 'U1'],
 ['2023-01-02', '2:35:00', '', '', '', '', '', 'U1'],
 ['2023-01-02', '3:05:00', '', '', '', '', '', 'U1'],
 ['2023-01-12', '1:05:00', '', '', '', '', '', 'U2'],
 ['2023-01-12', '1:05:00', '', '', '', '', '', 'U2'],
 ['2023-01-12', '1:05:00', '', '', '', '', '', 'U2'],
 ['2023-01-01', '1:05:00', '', '', '', '', '', 'U1'],
 ['2023-01-12', '1:35:00', '', '', '', '', '', 'U2']]

In [55]:
from pyspark.sql import functions as F
filepath=r"C:\Users\abhishek\Documents\coding\ds-algo\interview_test\publicis_sapient.txt"
data = spark.read.text(filepath).cache()

def someFunction(row):
    wordlist = row[0].split(" ")
    result = list()
    for word in wordlist:
        result.append(word.upper())
    return result

#rdd_df=data.rdd.map(someFunction).collect()
rdd_df=data.rdd.map(lambda r: r[0].split(" ")).zipWithIndex().toDF(['value', 'index'])
rdd_df.show(truncate=False)

+-----------------------------------+-----+
|value                              |index|
+-----------------------------------+-----+
|[]                                 |0    |
|[2023-01-01, 1:05:00, , , , , , u1]|1    |
|[2023-01-01, 1:25:00, , , , , , u1]|2    |
|[2023-01-02, 2:05:00, , , , , , u1]|3    |
|[2023-01-02, 2:35:00, , , , , , u1]|4    |
|[2023-01-02, 3:05:00, , , , , , u1]|5    |
|[2023-01-12, 1:05:00, , , , , , u2]|6    |
|[2023-01-12, 1:05:00, , , , , , u2]|7    |
|[2023-01-12, 1:05:00, , , , , , u2]|8    |
|[2023-01-01, 1:05:00, , , , , , u1]|9    |
|[2023-01-12, 1:35:00, , , , , , u2]|10   |
+-----------------------------------+-----+



In [63]:
rdd_df.select(F.regexp_replace('[',F.col("value"),"")).show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `[` cannot be resolved. Did you mean one of the following? [`index`, `value`].;
'Project [unresolvedalias(regexp_replace('[, value#528, , 1), Some(org.apache.spark.sql.Column$$Lambda$3317/0x000002612af295f8@ef159bc))]
+- LogicalRDD [value#528, index#529L], false


In [32]:
rdd_df.select(F.split(F.col("value"),", ")).show()

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "split(value, , , -1)" due to data type mismatch: Parameter 1 requires the "STRING" type, however "value" has the type "ARRAY<STRING>".;
'Project [unresolvedalias(split(value#460, , , -1), Some(org.apache.spark.sql.Column$$Lambda$3317/0x000002612af295f8@ef159bc))]
+- LogicalRDD [value#460, index#461L], false


In [9]:
spark.createDataFrame(rdd_df[2:]).show()

+----------+-------+---+---+---+---+---+---+
|        _1|     _2| _3| _4| _5| _6| _7| _8|
+----------+-------+---+---+---+---+---+---+
|2023-01-01|1:25:00|   |   |   |   |   | U1|
|2023-01-02|2:05:00|   |   |   |   |   | U1|
|2023-01-02|2:35:00|   |   |   |   |   | U1|
|2023-01-02|3:05:00|   |   |   |   |   | U1|
|2023-01-12|1:05:00|   |   |   |   |   | U2|
|2023-01-12|1:05:00|   |   |   |   |   | U2|
|2023-01-12|1:05:00|   |   |   |   |   | U2|
|2023-01-01|1:05:00|   |   |   |   |   | U1|
|2023-01-12|1:35:00|   |   |   |   |   | U2|
+----------+-------+---+---+---+---+---+---+



In [35]:
states = {"NY":"New York", "CA":"California", "FL":"Florida"}
broadcastStates = spark.sparkContext.broadcast(states)

data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]

rdd_df = spark.sparkContext.parallelize(data)

def state_convert(code):
    return broadcastStates.value[code]

result = rdd_df.map(lambda x: (x[0],x[1],x[2],state_convert(x[3]))).collect()
print(result)

[('James', 'Smith', 'USA', 'California'), ('Michael', 'Rose', 'USA', 'New York'), ('Robert', 'Williams', 'USA', 'California'), ('Maria', 'Jones', 'USA', 'Florida')]


In [ ]:
boolean_value={1:"True",0:"False"}
broadcastValue=spark.sparkContext.broadcast(boolean_value)

data=[()]


rdd_df=spark.sparkContext.paralleize(data)

In [39]:
dir(broadcastStates)

['__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_is_protocol',
 '_jbroadcast',
 '_path',
 '_pickle_registry',
 '_python_broadcast',
 '_sc',
 'destroy',
 'dump',
 'load',
 'load_from_path',
 'unpersist',
 'value']

In [52]:
accuvariable=spark.sparkContext.accumulator(2)
rdd_df=spark.sparkContext.parallelize(range(3,5))
rdd_df_1=rdd_df.foreach(lambda x:accuvariable.add(x))
print(accuvariable.value)

9
